In [3]:
import requests 
from bs4 import BeautifulSoup 
import csv 

# Send a GET request to the website 
page = requests.get('https://quotes.toscrape.com') 

# Parse the HTML content 
soup = BeautifulSoup(page.text, 'html.parser') 

# Create a list to store quotes 
quotes = [] 

# Find all quote elements 
quote_elements = soup.find_all('div', class_='quote') 

# Extract information from each quote element 
for quote_element in quote_elements: 
    # extract the text of the quote 
    text = quote_element.find('span', class_='text').text 
    
    # extract the author of the quote 
    author = quote_element.find('small', class_='author').text 

    # extract the tag <a> HTML elements related to the quote 
    tag_elements = quote_element.select('.tags .tag') 

    # store the list of tag strings in a list 
    tags = [] 
    for tag_element in tag_elements: 
        tags.append(tag_element.text) 
         
    quotes.append( 
        { 
            'text': text, 
            'author': author, 
            'tags': ', '.join(tags) # merge the tags into a "A, B, ..., Z" string 
        } 
    ) 
         
# Print the scraped quotes - optional 
for quote in quotes: 
    print("Quote: ", quote['text']) 
    print("Author: ", quote['author']) 
    print("Tags: ", quote['tags']) 
    print() 

# Save quotes to a CSV file 
with open('quotes.csv', 'w', encoding='utf-8', newline='') as csvfile: 
    # FIXED: Indented all of these lines so Python knows they belong to the open file
    fieldnames = ['text', 'author', 'tags'] 
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) 
    
    # Write headers 
    writer.writeheader() 
    
    # Write quotes 
    for quote in quotes: 
        # FIXED: Indented this so it happens inside the 'for' loop
        writer.writerow(quote) 

# FIXED: Removed the indentation so this only prints once at the very end!
print("Quotes have been saved to quotes.csv")

Quote:  “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Author:  Albert Einstein
Tags:  change, deep-thoughts, thinking, world

Quote:  “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Author:  J.K. Rowling
Tags:  abilities, choices

Quote:  “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Author:  Albert Einstein
Tags:  inspirational, life, live, miracle, miracles

Quote:  “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Author:  Jane Austen
Tags:  aliteracy, books, classic, humor

Quote:  “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Author:  Marilyn Monroe
Tags:  be-yourself, inspirational

Quote:  “Try not to become a man of success. Rather become a man of value.”
Author:  Albe

In [4]:
import requests 
from bs4 import BeautifulSoup 
import csv 
 
# Function to scrape quotes from a page 
def scrape_page(soup, quotes): 
    for quote in soup.find_all('div', class_='quote'): 
        text = quote.find('span', class_='text').text 
        author = quote.find('small', class_='author').text 
        tags = ', '.join(tag.text for tag in quote.find_all('a', class_='tag')) 
        quotes.append({'Text': text, 'Author': author, 'Tags': tags}) 
 
# Base URL and headers 
base_url = 'https://quotes.toscrape.com' 
headers = {'User-Agent': 'Mozilla/5.0'} 
 
# List to store quotes 
quotes = [] 
 
# Function to scrape quotes from multiple pages 
def scrape_all_pages(url): 
    while url: 
        response = requests.get(url, headers=headers) 
        soup = BeautifulSoup(response.text, 'html.parser') 
        scrape_page(soup, quotes) 
        next_page = soup.find('li', class_='next') 
        url = base_url + next_page.find('a')['href'] if next_page else None 
 
# Scrape quotes from all pages 
scrape_all_pages(base_url) 
 
# Save quotes to CSV file 
with open('quotes2.csv', 'w', newline='', encoding='utf-8') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames=['Text', 'Author', 'Tags']) 
    writer.writeheader() 
    writer.writerows(quotes)

In [5]:
import requests 
from bs4 import BeautifulSoup 
import csv 

# Base URL to combine with the "Next" button links
base_url = 'https://quotes.toscrape.com'
current_url = '/'  # Start at the home page

# Create a list to store the filtered quotes 
einstein_quotes = [] 

# Loop through pages as long as there is a "Next" button
while current_url:
    # Send a GET request to the current page
    page = requests.get(base_url + current_url) 
    soup = BeautifulSoup(page.text, 'html.parser') 
    
    # Find all quote elements on this page
    quote_elements = soup.find_all('div', class_='quote') 
    
    # Extract information from each quote element 
    for quote_element in quote_elements: 
        # Extract the author first so we can check the name
        author = quote_element.find('small', class_='author').text 
        
        # FILTER: Only process the rest if the author is Albert Einstein
        if author == 'Albert Einstein':
            text = quote_element.find('span', class_='text').text 
            
            # Extract the tags
            tag_elements = quote_element.select('.tags .tag') 
            tags = [tag.text for tag in tag_elements]  # A cleaner way to loop tags
            
            # Print to the console
            print("Quote: ", text) 
            print("Author: ", author) 
            print("Tags: ", ', '.join(tags)) 
            print("-" * 50) # Just a separator line to make it easy to read
            
            # Save it to our list
            einstein_quotes.append({ 
                'text': text, 
                'author': author, 
                'tags': ', '.join(tags) 
            }) 
            
    # Check if there is a "Next" button on the page
    next_button = soup.find('li', class_='next')
    
    if next_button:
        # If it exists, grab the link (e.g., '/page/2/') and loop again
        current_url = next_button.find('a')['href']
    else:
        # If no "Next" button is found, we've reached the last page!
        current_url = None 

# Save only the Einstein quotes to a new CSV file 
with open('quotes_Einstein.csv', 'w', encoding='utf-8', newline='') as csvfile: 
    fieldnames = ['text', 'author', 'tags'] 
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) 
    
    writer.writeheader() 
    for quote in einstein_quotes: 
        writer.writerow(quote) 

print(f"\nSuccess! All {len(einstein_quotes)} quotes by Albert Einstein have been saved to 'quotes_Einstein.csv'.")

Quote:  “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Author:  Albert Einstein
Tags:  change, deep-thoughts, thinking, world
--------------------------------------------------
Quote:  “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Author:  Albert Einstein
Tags:  inspirational, life, live, miracle, miracles
--------------------------------------------------
Quote:  “Try not to become a man of success. Rather become a man of value.”
Author:  Albert Einstein
Tags:  adulthood, success, value
--------------------------------------------------
Quote:  “If you can't explain it to a six year old, you don't understand it yourself.”
Author:  Albert Einstein
Tags:  simplicity, understand
--------------------------------------------------
Quote:  “If you want your children to be intelligent, read them fairy tales. If you want them to be more int